In [ ]:
%cd ../src

In [ ]:
%load_ext autoreload
%autoreload 2

import evaluator
from pathlib import Path
import seaborn as sns
import numpy as np

In [ ]:
evaluator_model_dict = {
    "model": "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic",
    "api_base": "***REMOVED***",
}
chatbot_model_dict = {
    "model": "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic",
    "api_base": "***REMOVED***",
}
analyzer_model_dict = {
    "model": "neuralmagic/Llama-3.1-Nemotron-70B-Instruct-HF-FP8-dynamic",
    "api_base": "***REMOVED***",
}


In [ ]:
factual_correctness_list = []
appropriateness_list = []
for i in range(1, 20):
    factual_correctness, appropriateness = evaluator.evaluate_scenario(i, evaluator_model_dict, chatbot_model_dict, analyzer_model_dict)
    factual_correctness_list.append(factual_correctness)
    appropriateness_list.append(appropriateness)
    print(factual_correctness_list, np.mean(np.array(factual_correctness_list)), np.std(np.array(factual_correctness_list)))
    print(appropriateness_list, np.mean(np.array(appropriateness_list)), np.std(np.array(appropriateness_list)))
    print("#"*20)